In [1]:
import pandas as pd
import numpy as np
     
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from nltk.tokenize import word_tokenize 
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
!pip install -q keras
import keras
from os import path
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(accelerator)
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.models import Sequential
from keras import layers
from google.colab import drive
drive.mount('/content/drive')
import csv



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


cpu
Mounted at /content/drive


## Data Reading

In [2]:
# """
# VALID MODE WİTH DOWNSAMPLING
# """

# train=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/TRAIN-TURK-ENCODED')
# test=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/TEST-TURK-ENCODED')

# X_train_FULL=train[['id','tweet_initial_nontoken','tweet','tweet_initial','subtask_a']]
# Y_TRAIN_ENCODED_FULL=train['subtask_a']
# X_test_FULL=test[['id','tweet_initial_nontoken','tweet','tweet_initial','subtask_a']]
# Y_TEST_ENCODED_FULL=test['subtask_a']

In [3]:
# 导入训练集数据 土耳其语
olid_training=pd.read_csv("/content/drive/MyDrive/TURKISH-DATA/offenseval-tr-training-v1.tsv",sep="\t")
X_train_FULL=olid_training[["id","tweet","subtask_a"]] 
Y_train_FULL=olid_training["subtask_a"]
# 导入测试数据及结果
X_test_FULL=pd.read_csv('/content/drive/MyDrive/TURKISH-DATA/offenseval-tr-testset-v1.tsv',sep='\t',encoding='utf8',quoting=csv.QUOTE_NONE)
Y_TEST=pd.read_csv('/content/drive/MyDrive/TURKISH-DATA/offenseval-tr-labela-v1.tsv',sep=',',encoding='utf8',quoting=csv.QUOTE_NONE,header=None)
Y_TRAIN_ENCODED_FULL=[1 if i ==  'OFF' else 0 for i in Y_train_FULL]
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_TEST[1]]

print("---导入数据成功---")



---导入数据成功---


In [15]:
# 导入训练集数据 英语
olid_training=pd.read_csv("/content/drive/MyDrive/OLIDv1.0/olid-training-v1.0.tsv",sep="\t")
X_train_FULL=olid_training[["id","tweet","subtask_a"]] 
Y_train_FULL=olid_training["subtask_a"]
# 导入测试数据及结果
X_test_FULL=pd.read_csv('/content/drive/MyDrive/OLIDv1.0/testset-levela.tsv',sep='\t',encoding='utf8',quoting=csv.QUOTE_NONE)
Y_TEST=pd.read_csv('/content/drive/MyDrive/OLIDv1.0/labels-levela.csv',sep=',',encoding='utf8',quoting=csv.QUOTE_NONE,header=None)
Y_TRAIN_ENCODED_FULL=[1 if i ==  'OFF' else 0 for i in Y_train_FULL]
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_TEST[1]]

print("---导入数据成功---")


---导入数据成功---


Pre-processing

In [16]:
# 数据清洗
X_train_FULL.tweet = X_train_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_train_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!='url' and w!=':' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)

X_train_FULL["tweet_initial"]=filtered_tweets


X_test_FULL.tweet = X_test_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_test_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!='url' and w!=':' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)

X_test_FULL["tweet_initial"] = filtered_tweets




#FOR TEST

z=[]
for tweet in X_train_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_train_FULL["tweet_initial_nontoken"]=z



#FOR TEST

z=[]
for tweet in X_test_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_test_FULL["tweet_initial_nontoken"]=z



In [6]:
X_test_FULL

,id,tweet,tweet_initial,tweet_initial_nontoken
0,15923,#whoisq #wherestheserver #dumpnike #declasfisa...,"[whoisq, wherestheserver, dumpnike, declasfisa...",whoisq wherestheserver dumpnike declasfisa dem...
1,27014,"#constitutionday is revered by conservatives, ...","[constitutionday, is, revered, by, conservativ...",constitutionday is revered by conservatives ha...
2,30530,#foxnews #nra #maga #potus #trump #2ndamendmen...,"[foxnews, nra, maga, potus, trump, 2ndamendmen...",foxnews nra maga potus trump 2ndamendment rnc ...
3,13876,#watching #boomer getting the news that she is...,"[watching, boomer, getting, the, news, that, s...",watching boomer getting the news that she is s...
4,60133,#nopasaran: unity demo to oppose the far-right...,"[nopasaran, unity, demo, to, oppose, the, far-...",nopasaran unity demo to oppose the far-right i...
...,...,...,...,...
855,73439,#despicabledems lie again about rifles. dem di...,"[despicabledems, lie, again, about, rifles, de...",despicabledems lie again about rifles dem dist...
856,25657,#meetthespeakers 🙌 @user will present in our e...,"[meetthespeakers, 🙌, will, present, in, our, e...",meetthespeakers 🙌 will present in our event oi...
857,67018,3 people just unfollowed me for talking about ...,"[3, people, just, unfollowed, me, for, talking...",3 people just unfollowed me for talking about ...
858,50665,#wednesdaywisdom antifa calls the right fascis...,"[wednesdaywisdom, antifa, calls, the, right, f...",wednesdaywisdom antifa calls the right fascist...


In [7]:
# 测试清洗后的数据
X_train_FULL = np.array(X_train_FULL)
X_test_FULL = np.array(X_test_FULL)
Y_TRAIN_ENCODED_FULL = np.array(Y_TRAIN_ENCODED_FULL)
Y_TEST_ENCODED_FULL = np.array(Y_TEST_ENCODED_FULL)


In [17]:
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers import Embedding
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
def recall_m(true_Y, pred_Y):
        TP = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        possible_pos = K.sum(K.round(K.clip(true_Y, 0, 1)))
        rec = TP / (possible_pos + K.epsilon())
        return rec

def precision_m(true_Y, pred_Y):
        true_positives = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(pred_Y, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision


    

def f1_m(true_Y, pred_Y):
    pres = precision_m(true_Y, pred_Y)
    rec = recall_m(true_Y, pred_Y)
    return 2*((pres*rec)/(pres+rec+K.epsilon()))

from keras.models import Sequential
from keras import layers


## Tweeter Word2vec / Custom Embedding Layer

In [ ]:
fname= "/content/drive/My Drive/Twitter/Word2Vec/w2v_model_word.vec"


import gensim
from gensim.models import Word2Vec
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(fname)  # you can load this saved keyedvectors model later



# Tweeter FastText

In [ ]:
import gensim
from gensim.models import FastText

word_vectors = gensim.models.FastText.load_fasttext_format('/content/drive/My Drive/Twitter/FastText/fastText_25022020.bin',encoding='utf-8') # use that if you want to use fasttedxt 



# Public FastText

In [ ]:
word_vectors = gensim.models.FastText.load_fasttext_format('/content/drive/My Drive/OFFENSEVAL20-DATA/haber-P1_S0_L0.bin',encoding='utf-8') # use that if you want to use fasttedxt 


# Public word2vec

In [9]:

fname= "/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz"

import gensim
from gensim.models import Word2Vec

word_vectors = gensim.models.KeyedVectors.load_word2vec_format(fname, binary = True)


## Tokenizing / creating vocabulary and wordindex using keras functinalities

In [18]:
"""
We will use word indexes as look-up table during embedding layer.
"""
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=98790)  #the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.
tokenizer.fit_on_texts(X_train_FULL["tweet_initial_nontoken"])
X_train_initial = tokenizer.texts_to_sequences(X_train_FULL["tweet_initial_nontoken"])
X_test_initial = tokenizer.texts_to_sequences(X_test_FULL["tweet_initial_nontoken"])
vocab_size_initial = len(tokenizer.word_index) + 1 
wordIndex_initial=tokenizer.word_index # it is  index
from keras_preprocessing.sequence import pad_sequences
max_len = 50

"""
Padding

"""
X_train_initial = pad_sequences(X_train_initial, padding='post', maxlen=max_len)
X_test_initial = pad_sequences(X_test_initial, padding='post', maxlen=max_len)




In [19]:
def createEmbeddingLayer(wordIndex,not_static):
  a=[]
  embedding_dim=300
  vocabulary_size=len(wordIndex)+1
  embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
  missed=0
  for word, i in wordIndex.items():
    
          
      try:
          embedding_vector = word_vectors[word] # or fast text
          embedding_matrix[i] = embedding_vector
            

      except KeyError: # If word is not found in the word2vec vocabulary , assign random weights
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_dim)
        missed+=1
        a.append(word)

  print('missed_words :' , missed)

  custom_embedding_layer = Embedding(vocabulary_size,
                                embedding_dim,
                                weights=[embedding_matrix],
                                trainable=not_static )# Controls the updating weights )
  return custom_embedding_layer

  

In [20]:
from sklearn.metrics import f1_score
import pickle
from keras.layers import GRU

def cnn(vocab_size,X_train,X_test,y_train,y_test,wordIndex,trainable):
  early_stopping = [EarlyStopping(monitor='val_loss',
                        min_delta=0,restore_best_weights=True,
                        patience=5,
                        verbose=1, mode='auto')]
  model = Sequential()
  model.add(createEmbeddingLayer(wordIndex,trainable))
  model.add(Dropout(0.2))
  model.add(GRU(32))
  #model.add(Conv1D(128, 1, activation='relu'))
  # model.add(Conv1D(128, 3, activation='relu'))
  # model.add(layers.GlobalMaxPooling1D())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  #model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
  # model.add(layers.Dense(100, activation='sigmoid'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False), metrics=['acc',f1_m,precision_m, recall_m])
  ## Fit the model
  model.fit(X_train, y_train, validation_split=0.1, epochs=20,callbacks=early_stopping,batch_size=32)
  loss, accuracy, f1_score, precision, recall = model.evaluate(X_train, y_train, verbose=1)
  print("cnn Training Loss: {:.4f}".format(loss))
  print("cnn Training Accuracy: {:.4f}".format(accuracy))
  print("cnn Training f1 score: {:.4f}".format(f1_score))
  print("cnn Training Precision: {:.4f}".format(precision))
  print("cnn Training Recall: {:.4f}".format(recall))

  loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)
  print("cnn Test Loss: {:.4f}".format(loss))
  print("cnn Test Accuracy: {:.4f}".format(accuracy))
  print("cnn Test f1 score: {:.4f}".format(f1_score))
  print("cnn Test Precision: {:.4f}".format(precision))
  print("cnn Test Recall: {:.4f}".format(recall))

  probs = model.predict(X_test, verbose=1)
  predicted_classes = (model.predict(X_test) > 0.5).astype("int32")


  #filename = 'finalized_model_lstm.sav'
  #pickle.dump(model, open(filename, 'wb'))

  print(classification_report(y_test, predicted_classes ,digits=3 ))


  print("cnn  ends..")
  return (predicted_classes,probs)




In [22]:
X_train_FULL = np.array(X_train_FULL)
X_test_FULL = np.array(X_test_FULL)
Y_TRAIN_ENCODED_FULL = np.array(Y_TRAIN_ENCODED_FULL)
Y_TEST_ENCODED_FULL = np.array(Y_TEST_ENCODED_FULL)

In [23]:
"""
Without Undersampled test results are in below "public w2v"
"""
prediction_cnnlstm,probs_cnnlstm=cnn(vocab_size_initial,X_train_initial,X_test_initial,Y_TRAIN_ENCODED_FULL,Y_TEST_ENCODED_FULL,wordIndex_initial,False)


missed_words : 5319
Epoch 1/20
373/373 [==============================] - 16s 35ms/step - loss: 0.6384 - acc: 0.6663 - f1_m: 0.0190 - precision_m: 0.0943 - recall_m: 0.0108 - val_loss: 0.6398 - val_acc: 0.6669 - val_f1_m: 0.0053 - val_precision_m: 0.0238 - val_recall_m: 0.0030
Epoch 2/20
373/373 [==============================] - 13s 34ms/step - loss: 0.6320 - acc: 0.6716 - f1_m: 0.0538 - precision_m: 0.2248 - recall_m: 0.0317 - val_loss: 0.6338 - val_acc: 0.6654 - val_f1_m: 0.0199 - val_precision_m: 0.1190 - val_recall_m: 0.0109
Epoch 3/20
373/373 [==============================] - 12s 33ms/step - loss: 0.5566 - acc: 0.7327 - f1_m: 0.4752 - precision_m: 0.5978 - recall_m: 0.4574 - val_loss: 0.4997 - val_acc: 0.7681 - val_f1_m: 0.6440 - val_precision_m: 0.6342 - val_recall_m: 0.6707
Epoch 4/20
373/373 [==============================] - 12s 33ms/step - loss: 0.4857 - acc: 0.7796 - f1_m: 0.6251 - precision_m: 0.7034 - recall_m: 0.5937 - val_loss: 0.4853 - val_acc: 0.7802 - val_f1_m: 0.63

In [ ]:
"""
Without Undersampled valid results are in below "public w2v"
"""
prediction_cnnlstm,probs_cnnlstm=cnn(vocab_size_initial,X_train_initial,X_test_initial,Y_TRAIN_ENCODED_FULL,Y_TEST_ENCODED_FULL,wordIndex_initial,False)


missed_words : 5319
Epoch 1/20
373/373 [==============================] - 14s 35ms/step - loss: 0.5804 - acc: 0.7078 - f1_m: 0.3549 - precision_m: 0.5373 - recall_m: 0.3069 - val_loss: 0.4925 - val_acc: 0.7734 - val_f1_m: 0.5995 - val_precision_m: 0.7294 - val_recall_m: 0.5324
Epoch 2/20
373/373 [==============================] - 15s 40ms/step - loss: 0.4757 - acc: 0.7765 - f1_m: 0.6031 - precision_m: 0.7212 - recall_m: 0.5485 - val_loss: 0.4802 - val_acc: 0.7795 - val_f1_m: 0.5817 - val_precision_m: 0.7853 - val_recall_m: 0.4767
Epoch 3/20
373/373 [==============================] - 12s 32ms/step - loss: 0.4349 - acc: 0.7986 - f1_m: 0.6488 - precision_m: 0.7507 - recall_m: 0.6054 - val_loss: 0.5370 - val_acc: 0.7364 - val_f1_m: 0.6443 - val_precision_m: 0.5694 - val_recall_m: 0.7656
Epoch 4/20
373/373 [==============================] - 13s 34ms/step - loss: 0.4004 - acc: 0.8166 - f1_m: 0.6870 - precision_m: 0.7730 - recall_m: 0.6512 - val_loss: 0.5000 - val_acc: 0.7795 - val_f1_m: 0.60

In [ ]:
"""
With Undersampled test results are in below "public w2v"
"""
prediction_cnnlstm,probs_cnnlstm=cnn(vocab_size_initial,X_train_initial,X_test_initial,Y_TRAIN_ENCODED_FULL,Y_TEST_ENCODED_FULL,wordIndex_initial,False)


missed_words : 17612
Train on 20787 samples, validate on 2310 samples
Epoch 1/20
20787/20787 [==============================] - 4s 175us/step - loss: 0.4757 - acc: 0.7904 - f1_m: 0.5349 - precision_m: 0.6740 - recall_m: 0.4973 - val_loss: 0.4632 - val_acc: 0.7978 - val_f1_m: 0.5572 - val_precision_m: 0.4667 - val_recall_m: 0.7500
Epoch 2/20
20787/20787 [==============================] - 3s 151us/step - loss: 0.3802 - acc: 0.8370 - f1_m: 0.6426 - precision_m: 0.7653 - recall_m: 0.5901 - val_loss: 0.3291 - val_acc: 0.8658 - val_f1_m: 0.4966 - val_precision_m: 0.6489 - val_recall_m: 0.4326
Epoch 3/20
20787/20787 [==============================] - 3s 155us/step - loss: 0.3398 - acc: 0.8512 - f1_m: 0.6753 - precision_m: 0.7887 - recall_m: 0.6327 - val_loss: 0.3310 - val_acc: 0.8654 - val_f1_m: 0.5440 - val_precision_m: 0.6523 - val_recall_m: 0.5109
Epoch 4/20
20787/20787 [==============================] - 3s 155us/step - loss: 0.3032 - acc: 0.8722 - f1_m: 0.7241 - precision_m: 0.8249 - reca

In [ ]:
"""
With Undersampled valid results are in below "public w2v"
"""
prediction_cnnlstm,probs_cnnlstm=cnn(vocab_size_initial,X_train_initial,X_test_initial,Y_TRAIN_ENCODED_FULL,Y_TEST_ENCODED_FULL,wordIndex_initial,False)


missed_words : 13202
Train on 15161 samples, validate on 1685 samples
Epoch 1/20
15161/15161 [==============================] - 3s 178us/step - loss: 0.4985 - acc: 0.7708 - f1_m: 0.5305 - precision_m: 0.6591 - recall_m: 0.5019 - val_loss: 0.4337 - val_acc: 0.8036 - val_f1_m: 0.6457 - val_precision_m: 0.6524 - val_recall_m: 0.6673
Epoch 2/20
15161/15161 [==============================] - 2s 147us/step - loss: 0.3877 - acc: 0.8298 - f1_m: 0.6584 - precision_m: 0.7722 - recall_m: 0.6127 - val_loss: 0.4279 - val_acc: 0.8160 - val_f1_m: 0.5902 - val_precision_m: 0.7794 - val_recall_m: 0.4945
Epoch 3/20
15161/15161 [==============================] - 2s 148us/step - loss: 0.3365 - acc: 0.8552 - f1_m: 0.7114 - precision_m: 0.8075 - recall_m: 0.6721 - val_loss: 0.4265 - val_acc: 0.8172 - val_f1_m: 0.6213 - val_precision_m: 0.7596 - val_recall_m: 0.5477
Epoch 4/20
15161/15161 [==============================] - 2s 154us/step - loss: 0.2905 - acc: 0.8758 - f1_m: 0.7568 - precision_m: 0.8380 - reca

In [ ]:
cnnlstm_test_full_result= pd.DataFrame()
cnnlstm_test_full_result['CNN-LSTM-PREDICTON']=list(np.squeeze(prediction_cnnlstm))
cnnlstm_test_full_result['CNN-LSTM-PROBS']=probs_cnnlstm
cnnlstm_test_full_result['Gold-label']=Y_TEST_ENCODED_FULL



In [ ]:
cnnlstm_test_full_result

,CNN-LSTM-PREDICTON,CNN-LSTM-PROBS,Gold-label
0,0,0.176501,1
1,0,0.309252,0
2,0,0.260592,0
3,0,0.106699,0
4,0,0.096021,1
...,...,...,...
855,0,0.164990,1
856,0,0.088237,0
857,0,0.392903,1
858,0,0.291047,0


In [ ]:
cnnlstm_test_full_result.to_csv('cnnlstm_test_full_result.csv',header=False)

In [ ]:
cnnlstm_valid_full_result= pd.DataFrame()
cnnlstm_valid_full_result['CNN-LSTM-PREDICTON']=list(np.squeeze(prediction_cnnlstm))
cnnlstm_valid_full_result['CNN-LSTM-PROBS']=probs_cnnlstm
cnnlstm_valid_full_result['Gold-label']=Y_TEST_ENCODED_FULL

cnnlstm_valid_full_result.to_csv('cnnlstm_valid_full_result.csv',header=False)

In [ ]:
cnnlstm_valid_full_result.head(20)

,CNN-LSTM-PREDICTON,CNN-LSTM-PROBS,Gold-label
0,0,0.176501,1
1,0,0.309252,0
2,0,0.260592,0
3,0,0.106699,0
4,0,0.096021,1
5,1,0.916323,1
6,0,0.313791,0
7,0,0.115352,1
8,0,0.081312,0
9,0,0.079693,0
